In [2]:
from numpy import power, array
from pybary import bary_recursive

oracle = lambda x: power(x, 2)
x0 = array([0, 0])

x = bary_recursive(oracle, x0, 10, 0.1, 0, 1, 100)

print(x)

[[-0.04252469  0.06124647]]


In [15]:
from numpy import power, array
from numpy.random import normal

from pybary import bary_recursive

oracle = lambda x: power(x, 2)
xs = normal(0, 1, [100, 2])

x = bary_recursive(oracle, xs, 10, 0.1)

print(x)

[0.1368491  0.31666112]
